In [69]:
import pandas as pd
import os
import openai
from sklearn.metrics import accuracy_score

from tenacity import (
retry,
stop_after_attempt,
wait_random_exponential,
)

In [17]:
os.getcwd()

'C:\\Users\\saran\\Desktop\\LLM Seminar\\Apps Phase\\LLM_Data_Annotation'

In [ ]:
os.chdir('..')

In [70]:
data = pd.read_csv('data/sentiment/annotated/gpt35/conf_scores_1000.csv', index_col = 0 )

In [78]:
data[data.predicted_labels != "the sentiment of the text is 'positive'."].iloc[0:999].reset_index(drop = True, inplace = True)

In [80]:
data[data.predicted_labels != "the sentiment of the text is 'positive'."].iloc[0:999].to_csv('data/sentiment/annotated/gpt35/conf_scores_1000_preproc.csv')

In [77]:
data[data.confidence_score <1 ]

,text,predicted_labels,confidence_score
5,http://www.dothebouncy.com/smf - some shameles...,positive,0.666667
16,"S`ok, trying to plot alternatives as we speak...",negative,0.666667
22,is cleaning the house for her family who is co...,positive,0.333333
27,On the way to Malaysia...no internet access to...,neutral,0.333333
34,"Ahhh, I slept through the game. I`m gonna tr...",positive,0.666667
...,...,...,...
913,has broken off the fb wedding so sadly no long...,negative,0.666667
923,cavs got lucky 2night lol but lebron took ove...,the sentiment of the text is 'positive'.,0.333333
933,Buried under more web changes. Going to make l...,negative,0.666667
978,Ha! Thanks Bryan! And don`t remind me about t...,positive,0.333333


In [81]:
data2 = pd.read_csv('data/sentiment/annotated/gpt35/conf_scores_1000_preproc.csv', index_col=0)

In [63]:
unannotated = pd.read_csv('data/sentiment/unannotated/unannotated_sentiment_dataset.csv', encoding= 'unicode_escape', index_col=[0])
original_dataset = pd.read_csv('data/sentiment/original/train.csv', encoding= 'unicode_escape')

In [64]:
with open('openai/organization.txt', 'r') as file:
    openai.organization = file.read().strip()

with open('openai/key.txt', 'r') as file:
    openai.api_key = file.read().strip()

In [36]:
accumulated_tokens_method1 = 0
accumulated_cost_method1 = 0
cost_per_token = 0.0035 / 1000  # The total cost per token, input and output
index = 0

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def analyze_ada(text):
    global index
    global accumulated_tokens_method1
    global accumulated_cost_method1

    prompt = f"Sentiment analysis for the following text in a single number: 1 for positive, 0 for neutral, 2 for negative: \"{text}\""

    response = openai.Completion.create(
        engine="text-ada-001",
        prompt=prompt,
        max_tokens=10,
        temperature=0
    )

    total_tokens_used = response['usage']['total_tokens']
    print(f"Total tokens used for this call: {total_tokens_used}")

    call_cost = total_tokens_used * cost_per_token
    accumulated_cost_method1 += call_cost
    accumulated_tokens_method1 += total_tokens_used
    index += 1
    print('\nIndex: ', index)
    print(f"Cost for this call: {call_cost}")
    print(f"Accumulated tokens so far: {accumulated_tokens_method1}")
    print(f"Accumulated cost so far: {accumulated_cost_method1}")

    response_text = response.choices[0].text.strip().lower()

    return response_text


In [65]:
accumulated_tokens = 0
accumulated_cost = 0
cost_per_token = 0.0035 / 1000  # The total cost per token, input and output
index = 0

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def analyze_gpt35(text):
    global index
    global accumulated_cost
    global accumulated_tokens
    messages = [
        {"role": "system", "content": "Your task is to analyze text and classify its sentiment as either 'positive', 'negative', or 'neutral' in a single word."},
        {"role": "user", "content": f"Classify the sentiment of: '{text}'."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=10,
        n=3,
        stop=None,
        temperature=0.5
    )

    total_tokens_used = response['usage']['total_tokens']
    print(f"Total tokens used for this call: {total_tokens_used}")

    call_cost = total_tokens_used * cost_per_token
    accumulated_cost += call_cost
    accumulated_tokens += total_tokens_used
    index += 1
    print('Index: ', index)
    print(f"Cost for this call: {call_cost}")
    print(f"Accumulated tokens so far: {accumulated_tokens}")
    print(f"Accumulated cost so far: {accumulated_cost}\n")

    response_texts = [choice.message.content.strip().lower() for choice in response.choices]
    primary_response = response_texts[0]
    confidence_score = response_texts.count(primary_response) / 3

    return primary_response, confidence_score

In [66]:
llm_annotated_data = unannotated.copy()

In [68]:
num_rows = 20

In [40]:
#llm_annotated_data['predicted_gpt35'] = llm_annotated_data['text'].iloc[0:num_rows].apply(analyze_gpt35)
#llm_annotated_data['predicted_label'] = llm_annotated_data['text'].iloc[0:num_rows].apply(analyze_ada)

Total tokens used for this call: 46

Index:  1
Cost for this call: 0.000161
Accumulated tokens so far: 46
Accumulated cost so far: 0.000161
Total tokens used for this call: 42

Index:  2
Cost for this call: 0.000147
Accumulated tokens so far: 88
Accumulated cost so far: 0.000308
Total tokens used for this call: 34

Index:  3
Cost for this call: 0.00011899999999999999
Accumulated tokens so far: 122
Accumulated cost so far: 0.00042699999999999997
Total tokens used for this call: 42

Index:  4
Cost for this call: 0.000147
Accumulated tokens so far: 164
Accumulated cost so far: 0.000574
Total tokens used for this call: 46

Index:  5
Cost for this call: 0.000161
Accumulated tokens so far: 210
Accumulated cost so far: 0.000735
Total tokens used for this call: 55

Index:  6
Cost for this call: 0.0001925
Accumulated tokens so far: 265
Accumulated cost so far: 0.0009274999999999999
Total tokens used for this call: 46

Index:  7
Cost for this call: 0.000161
Accumulated tokens so far: 311
Accumul

In [52]:
sentiments_and_scores = llm_annotated_data['text'].iloc[0:num_rows].apply(analyze_gpt35)
llm_annotated_data.loc[llm_annotated_data.index[0:num_rows], 'predicted_labels'] = [x[0] for x in sentiments_and_scores]
llm_annotated_data.loc[llm_annotated_data.index[0:num_rows], 'confidence_score'] = [x[1] for x in sentiments_and_scores]
llm_annotated_data['annotation_correct'] = (llm_annotated_data['predicted_labels'] == original_dataset['sentiment']).astype(str)

{
  "id": "chatcmpl-7l4a5IC8NPvLQ4Hxhame0VcnFWJk1",
  "object": "chat.completion",
  "created": 1691453381,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "neutral"
      },
      "finish_reason": "stop"
    },
    {
      "index": 1,
      "message": {
        "role": "assistant",
        "content": "neutral"
      },
      "finish_reason": "stop"
    },
    {
      "index": 2,
      "message": {
        "role": "assistant",
        "content": "neutral"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 56,
    "completion_tokens": 3,
    "total_tokens": 59
  }
}
Total tokens used for this call: 59
Index:  1
Cost for this call: 0.0002065
Accumulated tokens so far: 59
Accumulated cost so far: 0.0002065
{
  "id": "chatcmpl-7l4a5VPvz1pcRiuZ9xGOFqxPqTmnZ",
  "object": "chat.completion",
  "created": 1691453381,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
 

In [61]:
llm_annotated_data

,text,predicted_labels,confidence_score,annotation_correct
0,"I`d have responded, if I were going",neutral,1.0,1
1,Sooo SAD I will miss you here in San Diego!!!,negative,1.0,1
2,my boss is bullying me...,negative,1.0,1
3,what interview! leave me alone,negative,1.0,1
4,"Sons of ****, why couldn`t they put them on t...",negative,1.0,1
...,...,...,...,...
27476,wish we could come see u on Denver husband l...,NaN,NaN,0
27477,I`ve wondered about rake to. The client has ...,NaN,NaN,0
27478,Yay good for both of you. Enjoy the break - y...,NaN,NaN,0
27479,But it was worth it ****.,NaN,NaN,0


In [43]:
#llm_annotated_data.to_csv('data/sentiment/annotated/ada/ada_500.csv')

In [55]:
#original_dataset.loc[0:num_rows - 1, 'sentiment'] = original_dataset['sentiment'].iloc[0:num_rows].map(sentiments)

In [60]:
#original_dataset

In [57]:
llm_annotated_data.iloc[0:num_rows]

,text,predicted_labels,confidence_score,annotation_correct
0,"I`d have responded, if I were going",neutral,1.000000,1
1,Sooo SAD I will miss you here in San Diego!!!,negative,1.000000,1
2,my boss is bullying me...,negative,1.000000,1
3,what interview! leave me alone,negative,1.000000,1
4,"Sons of ****, why couldn`t they put them on t...",negative,1.000000,1
5,http://www.dothebouncy.com/smf - some shameles...,positive,1.000000,0
6,2am feedings for the baby are fun when he is a...,positive,1.000000,1
7,Soooo high,positive,1.000000,0
8,Both of you,neutral,1.000000,1
9,Journey!? Wow... u just became cooler. hehe....,positive,1.000000,1


In [59]:
#llm_annotated_data['annotation_correct'] = (llm_annotated_data['predicted_gpt35'].iloc[0:num_rows] == original_dataset['sentiment'].iloc[0:num_rows]).astype(int)

llm_annotated_data['annotation_correct'] = (llm_annotated_data['predicted_labels'].iloc[0:num_rows].astype(int) == original_dataset['sentiment'].iloc[0:num_rows].astype(int)).astype(int)

ValueError: invalid literal for int() with base 10: 'neutral'

In [ ]:
llm_annotated_data

In [ ]:
llm_annotated_data.iloc[0:num_rows].to_csv('data/sentiment/gpt35_annotated.csv')

In [ ]:
#llm_annotated_data['predicted_gpt35'] = llm_annotated_data['text'].iloc[0:num_rows].apply(analyze_gpt35)

In [ ]:
#print(f"Accuracy of GPT3.5's annotations: {accuracy_score(original_dataset['sentiment'].iloc[0:num_rows].astype('str').values, llm_annotated_data['predicted_gpt35'].iloc[0:num_rows].astype('str').values)}")

print(f"Accuracy of Davinci 003's annotations: {accuracy_score(original_dataset['sentiment'].iloc[0:num_rows].astype('str').values, llm_annotated_data['predicted_davinci'].iloc[0:num_rows].astype('str').values)}")

In [ ]:
#llm_annotated_data.iloc[0:num_rows].to_csv('data/sentiment/gpt35_annotated.csv')

In [ ]:
dataset = pd.read_csv('data/sentiment/davinci003_annotated_300.csv', index_col=[0])

In [ ]:
dataset['predicted_davinci'] = dataset['predicted_davinci'].apply(lambda x: x.replace('.', ''))

In [ ]:
dataset['predicted_davinci'].value_counts()

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [ ]:
MODEL_NAME = 'bert-base-uncased'
BATCH_SIZE = 16
EPOCHS = 3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data = dataset.copy()
sentiments = {'positive': 0, 'neutral': 1, 'negative': 2}
data['predicted_davinci'] = data['predicted_davinci'].map(sentiments)

In [ ]:
data['predicted_davinci']

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
train_texts, val_texts, train_targets, val_targets = train_test_split(data['text'], data['predicted_davinci'], test_size=0.1)

train_texts = train_texts.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
train_targets = train_targets.reset_index(drop=True)
val_targets = val_targets.reset_index(drop=True)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

train_data = SentimentDataset(train_texts, train_targets, tokenizer, max_len=128)
val_data = SentimentDataset(val_texts, val_targets, tokenizer, max_len=128)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(sentiments)).to(DEVICE)

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
def train_epoch(model, data_loader, optimizer, device, scheduler=None):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == targets)

        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        if scheduler:
            scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

def eval_model(model, data_loader, device):
    model = model.eval()

    correct_predictions = 0
    predictions = []
    real_values = []

    with torch.no_grad():
        for d in tqdm(data_loader):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
            _, preds = torch.max(outputs.logits, dim=1)

            predictions.extend(preds)
            real_values.extend(targets)
            correct_predictions += torch.sum(preds == targets)

    predictions = torch.stack(predictions).cpu()
    real_values = torch.stack(real_values).cpu()
    return correct_predictions.double() / len(data_loader.dataset), classification_report(real_values, predictions, target_names=sentiments.keys())

In [ ]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(model, train_loader, optimizer, DEVICE)
    print(f'Train loss: {train_loss}, accuracy: {train_acc}')

    val_acc, val_report = eval_model(model, val_loader, DEVICE)
    print(f'Val accuracy: {val_acc}\n')
    print(val_report)

In [ ]:
torch.save(model.state_dict(), 'models/davinci/bert_sentiment_davinci003_300_model.pt')

In [ ]:
test = pd.read_csv('data/sentiment/test.csv', encoding= 'unicode_escape')

In [ ]:
test.drop(columns=[x for x in test.columns if x != 'text' and x != 'sentiment'], inplace=True)

In [ ]:
test.dropna(subset=['sentiment'], inplace=True)

In [ ]:
test.sentiment.value_counts()

In [ ]:
for i in test.columns:
    test[i] = test[i].astype('str')

In [ ]:
test_texts = test['text'].reset_index(drop=True)
test_targets = test['sentiment'].reset_index(drop=True)

In [ ]:
if isinstance(test_targets[0], str):
    label_encoder = LabelEncoder()
    test_targets = label_encoder.fit_transform(test_targets)

In [ ]:
test_data = SentimentDataset(test_texts, test_targets, tokenizer, max_len=128)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
test_acc, test_report = eval_model(model, test_loader, DEVICE)
print(f'Test accuracy: {test_acc}')
print(test_report)